In [1]:
# Input window length, we should be able to genearate a graph assuming k=2
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
import functools
import re
import random
from collections import Counter

In [2]:
def turn_tuple_to_str(tuple):
    string = ''
    for element in tuple:
        string += str(element)
    return string

In [3]:
def generate_vertex(k, n):
    """This function inputs an alphabet k and window size n, and output all nodes in the de Bruijn graph
    >>>generate_vertex([0,1], 4)
    >>>[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]"""
    length = n-1
    lst = list(itertools.product(k, repeat=length))
    return lst

In [4]:
def generate_edges(k, n):
    """
    >>> generate_edges(['0', '1'], 4)
    >>> ['000----1--->001', '100----0--->000', '001----0--->010', '001----1--->011', '100----1--->001', '010----0--->100', '010----1--->101', '101----0--->010', '101----1--->011', '011----0--->110', '011----1--->111', '110----0--->100', '110----1--->101', '111----0--->110', '000----0--->000', '111----1--->111']
    """
    v_lst = generate_vertex(k,n)
    edge_lst = []
    for v1, v2 in itertools.combinations(v_lst, 2):
        node1_copy = v1[1:]
        node2_copy = v2[:n-2]
        label1 = v2[n-2]
        node1_copy2 = v1[:n-2]
        node2_copy2 = v2[1:]
        label2 = v1[n-2]
        if node1_copy == node2_copy:
            edge_lst.append(turn_tuple_to_str(v1)+ "----" + label1 + "--->" + turn_tuple_to_str(v2))
        if node1_copy2 == node2_copy2:
            edge_lst.append(turn_tuple_to_str(v2)+ "----" + label2 + "--->" + turn_tuple_to_str(v1))
    for element in k:
        edge_lst.append((element*(n-1)) + "----" + element + "--->"+ (element*(n-1)))
    return edge_lst

In [5]:
def eulerian_path(k,n):
    edge_lst = generate_edges(k,n)
    new_lst = []
    for item in edge_lst:
        start_vertex = item[:n-1]
        end_vertex = item[-n+1:]
        new_lst.append((start_vertex, end_vertex))
    random.shuffle(new_lst)
    G = nx.DiGraph()
    G.add_edges_from(new_lst)
    return [u for u, v in nx.eulerian_circuit(G)]

In [6]:
def generate_debruijn(k, n: int) -> str:
    vertex_lst = eulerian_path(k,n)
    str = ''
    for vertex in vertex_lst:
        str += vertex[-1]
    return str

In [7]:
generate_debruijn(['a','b','c'],4)

'ababcbcbbcbabaabacbaccbaacbbaaaabcacbcabccbbbbabbbcaaaccacccbccaabbccccabbacaacac'

In [8]:
def summary(k, n):
    v_lst = generate_vertex(k,n)
    e_lst = generate_edges(k,n)
    print ("You have entered an alphabet of " + str(k) + ", and you are computing for window length " + str(n))
    print ("The number of vertices is " + str(len(v_lst)) + ", they are " + str(v_lst))
    print ("The number of edges is " + str(len(e_lst)) + ", they are " + str(e_lst))
    print ("Its eulerian circuit is " + str(eulerian_path(k,n)))
    print ("And the de Bruijn sequence is " + str(generate_debruijn(k,n)))

In [10]:
summary(['2','1','0'],3)

You have entered an alphabet of ['2', '1', '0'], and you are computing for window length 3
The number of vertices is 9, they are [('2', '2'), ('2', '1'), ('2', '0'), ('1', '2'), ('1', '1'), ('1', '0'), ('0', '2'), ('0', '1'), ('0', '0')]
The number of edges is 27, they are ['22----1--->21', '22----0--->20', '12----2--->22', '02----2--->22', '21----2--->12', '12----1--->21', '21----1--->11', '21----0--->10', '02----1--->21', '12----0--->20', '20----2--->02', '02----0--->20', '20----1--->01', '20----0--->00', '11----2--->12', '01----2--->12', '11----0--->10', '01----1--->11', '10----2--->02', '10----1--->01', '01----0--->10', '10----0--->00', '00----2--->02', '00----1--->01', '22----2--->22', '11----1--->11', '00----0--->00']
Its eulerian circuit is ['22', '20', '02', '21', '11', '11', '12', '21', '12', '20', '00', '00', '02', '20', '01', '11', '10', '00', '01', '12', '22', '21', '10', '01', '10', '02', '22']
And the de Bruijn sequence is 220001110100210202221120121


In [1]:
def get_as_many_bruijn(k,n):
    total_num = (math.factorial(len(k))**(len(k)**(n-1))/(len(k)**n))
    print ("in total " + str(total_num) + " de Bruijn sequences")
    output = []
    while len(output) < 100:
        sequence = generate_debruijn(k,n)
        if sequence not in output:
            output.append(sequence)
    return output

In [11]:
def valid_4d_sequence(lst1,lst2,lst3,lst4):
    """these 4 lists are 4 de Bruijn sequence of the same length, we need to check if it is valid in the context of the game SET"""
    compare_item = []
    for i in range(len(lst1)):
        if [lst1[i],lst2[i],lst3[i],lst4[i]] not in compare_item:
            compare_item.append([lst1[i],lst2[i],lst3[i],lst4[i]])
        else:
            return False
    return True

In [12]:
def find_valid_sequence(lst):
    """input a list in the format of the output in get_as_many_debruijn"""
    i = 0
    for lst1, lst2, lst3, lst4 in itertools.combinations(lst, 4):
        i += 1
        if valid_4d_sequence(lst1,lst2,lst3,lst4):
            print ([lst1,lst2,lst3,lst4])
            return True
        if i%1000000 == 0:
            print (i)
    return False

In [13]:
def shift_sequence(sequence, position):
    return sequence[-position:] + sequence[:len(sequence)-position]

In [14]:
def valid_2d_sequence(lst1,lst2):
    """these 4 lists are 4 de Bruijn sequence of the same length, we need to check if it is valid in the context of the game SET"""
    compare_item = []
    for i in range(len(lst1)):
        compare_item.append([lst1[i],lst2[i]])
    if compare_item.count(['a','a']) == compare_item.count(['b','b']) == compare_item.count(['c','c']) == compare_item.count(['a','b']) == compare_item.count(['a','c']) == compare_item.count(['b','a']) == compare_item.count(['c','a']) == compare_item.count(['b','c']) == compare_item.count(['c','b']) == 9:
        return True
    return False

In [15]:
def valid_2d_sequence(lst1,lst2,lst3):
    """these 4 lists are 4 de Bruijn sequence of the same length, we need to check if it is valid in the context of the game SET"""
    compare_item = []
    for i in range(len(lst1)):
        compare_item.append([lst1[i],lst2[i],lst3[i]])
    if compare_item.count(['a','a']) == compare_item.count(['b','b']) == compare_item.count(['c','c']) == compare_item.count(['a','b']) == compare_item.count(['a','c']) == compare_item.count(['b','a']) == compare_item.count(['c','a']) == compare_item.count(['b','c']) == compare_item.count(['c','b']) == 9:
        return True
    return False

In [16]:
def find_valid_by_shifting(seq):
    # idea: if we look at 2 sequence, vertically it must have the same number of aa, bb, cc, ab, ba, ac, ca, bc, cb, each one appears 81/9 = 9 times
    shifted_seq1 = shift_sequence(seq, 80)
    shifted_seq2 = shift_sequence(seq, 79)
    shifted_seq3 = shift_sequence(seq, 78)
    if valid_4d_sequence(seq, shifted_seq1, shifted_seq2, shifted_seq3):
        return [seq, shifted_seq1, shifted_seq2, shifted_seq3]

In [17]:
"""output = get_as_many_bruijn(['a','b','c'], 4)"""

"output = get_as_many_bruijn(['a','b','c'], 4)"

In [18]:
"""lst = []
for debruijn in output:
    lst.append(find_valid_by_shifting(debruijn))"""

'lst = []\nfor debruijn in output:\n    lst.append(find_valid_by_shifting(debruijn))'

In [19]:
"""len(lst)"""

'len(lst)'

In [20]:
lst1='acbccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcba'
lst2='bccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcbaac'
lst3='ccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcbaacb'
lst4='cbccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcbaa'
valid_4d_sequence(lst1,lst2,lst3,lst4)

True

In [21]:
def answer_1(seq):
    lst_1 = [0,0,0,0]
    convert = list(seq)
    for i in range(len(convert)):
        if convert[i] == 'a':
            lst_1[i]=1
    return lst_1

In [22]:
def answer_2(seq):
    lst_1 = [0,0,0,0]
    convert = list(seq)
    for i in range(len(convert)):
        if convert[i] == 'b':
            lst_1[i]=1
    return lst_1

In [23]:
lst = ['a','c','b','c', 'd', 'c', 's']
lst[0:4]

['a', 'c', 'b', 'c']

In [24]:
def count_answer(str_1, str_2):
    big_lst = []
    lst_1 = list(str_1)
    lst_2 = list(str_2)
    for i in range(len(str_1)-3):
        seq_1 = lst_1[i:i+4]
        ans_1 = answer_1(seq_1)
        seq_2 = lst_2[i:i+4]
        ans_2 = answer_2(seq_2)
        combine = ans_1 + ans_2
        if combine not in big_lst:
            big_lst.append(combine)
    ans_1 = answer_1(lst_1[78:81] + [lst1[0]])
    ans_2 = answer_2(lst_2[78:81] + [lst2[0]])
    combine = ans_1 + ans_2
    if combine not in big_lst:
        big_lst.append(combine)
    ans_1 = answer_1(lst_1[79:81] + lst_1[0:2])
    ans_2 = answer_2(lst_2[79:81] + lst_2[0:2])
    combine = ans_1 + ans_2
    if combine not in big_lst:
        big_lst.append(combine)
    ans_1 = answer_1(lst_1[80:81] + lst_1[0:3])
    ans_2 = answer_2(lst_2[80:81] + lst_2[0:3])
    combine = ans_1 + ans_2
    if combine not in big_lst:
        big_lst.append(combine)
    if len(big_lst) == 81:
        print (str_1)
    return len(big_lst)

In [34]:
plot_lst = []
possible = []
for sequence in get_as_many_bruijn(['a','b','c'],4):
    shifted_1 = shift_sequence(sequence, 80)
    shifted_2 = shift_sequence(sequence, 79)
    shifted_3 = shift_sequence(sequence, 78)
    possible.append(count_answer(sequence, shifted_1))
    possible.append(count_answer(sequence, shifted_2))
    possible.append(count_answer(sequence, shifted_3))
    possible.append(count_answer(shifted_1, shifted_2))
    possible.append(count_answer(shifted_1, shifted_3))
    possible.append(count_answer(shifted_2, shifted_3))
    plot_lst.append(min(possible))
"""counted = Counter(plot_lst)
for item in counted:
    plt.bar(item, counted[item])
plt.show()"""

#conversion for stats calculation
print (plot_lst.count(79))
print (plot_lst.count(80))
print (plot_lst.count(81))
print (max(plot_lst))
print ('mean: ' + str(np.mean(plot_lst)))
print ('median: ' + str(np.median(plot_lst)))
print ('standard deviation: ' + str(np.std(plot_lst)))

in total 1.2635683568857645e+19 de Bruijn sequences


baccbacaaaaccccbcbababcbccbbbbabbbcbbacbcaacbaacacabcaccacbbcabaaabcccabbccaabbaa
accbacaaaaccccbcbababcbccbbbbabbbcbbacbcaacbaacacabcaccacbbcabaaabcccabbccaabbaab
ccbacaaaaccccbcbababcbccbbbbabbbcbbacbcaacbaacacabcaccacbbcabaaabcccabbccaabbaaba


baccaaaacacaaccccaccbcbcccbbbbcbbccbacabccababcbabaacbcabbbabbcacbbacbaaabcaabbaa
accaaaacacaaccccaccbcbcccbbbbcbbccbacabccababcbabaacbcabbbabbcacbbacbaaabcaabbaab
ccaaaacacaaccccaccbcbcccbbbbcbbccbacabccababcbabaacbcabbbabbcacbbacbaaabcaabbaaba


aaaaccaccccaacacaaabccabcaabacababcbcbbbbcbaccbcccbbccbabaacbcacbaabbcabbbacbbabb


baaaacaaabcaacbacacabcbabccaabbcabbacbcacbbcbcbbabbbccbccccbbbbaabaacccababaccacc


...

bccccbbbbccbcbbcbcaacaaaacccaabbcabbbaaabaacbbabbacababaccbaabccabcbabcacaccacbac


aacbccccbbccbcbbbbcbcabbcaaaacababaacccabccaabaccbabcbacbaaabcacaccacbbabbbaabbac


baaaaccaccccababaccbcbcccbbcbbbbccbaacacabcbabccaacbcabbcacbacbbabbbacaaabbaabcaa


baaaacacaaabacababaacbcacbaabcaaccccbcbcccaccaabbcabccabbaccbacbbccbbabcbabbbcbbb


0
0
0
67
mean: 48.5551
median: 47.0
standard deviation: 1.8381740913199707


In [1]:
a = 'acbccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcba'
b = 'cbccccbcbcabcaacabbacaaaacccaabccabacbbaaabaabbbababbccbbcacacbaccaccbabcbbbbcbaa'
count_answer(a,b)

NameError: name 'count_answer' is not defined